This notebook performs an error analysis of our block stitching process using IoU tracking across interface slices. The idea is to characterize error statistics of source block and target block before linking, and of the combined superblock after linking. Ideally, the ERL of the superblock should be as close as possible to the sum of ERLs of source and target blocks.

The code below assumes source and target blocks have been apropriately generated using the SegPrep module.

## Block stitching

In [1]:
from cerebellum.data_prep.seg_prep import *
from cerebellum.error_analysis.skel_segeval import *
import json

# set params
resolution = (30, 48, 48)
block_size = 60
affinity_offset = 14
wz_thresh = 0.5
sblock_id = 0 # source block id
tblock_id = 1 # target block id

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# names of source and target block segmentations
zz_sb = sblock_id*block_size + affinity_offset
gt_sb_name = "gt-48nm-%dslices-%04d"%(block_size, zz_sb)
pred_sb_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz_sb)
zz_tb = tblock_id*block_size + affinity_offset
tblock_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz_tb)
gt_tb_name = "gt-48nm-%dslices-%04d"%(block_size, zz_tb)
pred_tb_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz_tb)

In [3]:
# skeleton error analysis thresholds
t_om = 0.9
t_m = 0.5
t_s = 0.8

In [4]:
# run error analysis of source block
sb_vox_eval = VoxEval(gt_sb_name, pred_sb_name, stage="filtered")
sb_vox_eval.run_fullsuite(iou_max=0.6, hist_segs=10, 
                          thresh_miss=t_om, overwrite_prev=False)
sb_skel_eval = SkelEval(gt_sb_name, pred_sb_name, dsmpl_res=(80,80,80), 
	                 t_om=t_om, t_m=t_m, t_s=t_s, 
	                 include_zero_split=False, include_zero_merge=True,
	                 stage="filtered", overwrite_prev=True)
sb_skel_eval.merge_oracle()
sb_skel_eval.split_oracle()

Starting voxel evaluation methods for waterz0.50-48nm-crop2gt-0014 against gt-48nm-60slices-0014
Missing object results loaded
VI loaded
IoU results loaded
delta_VI results loaded
Delta VI histogram plots generated and saved
Starting error analysis of waterz0.50-48nm-crop2gt-0014 against skeletons of gt-48nm-60slices-0014
Warning! Labels in predicted segmentation are not consecutive. Proceeding anyway
Starting evaluation of 5738 labels in 60x540x489 predicted segmentation against 1003 GT skeletons
Using error thresholds: t_om=0.90, t_m=0.50, t_s=0.80
Skeleton evaluation time: 3.51039600372
Results:
23 omissions, 15 merges, 126 splits, 12 hybrid, 827 correct
GT ERL: 2005, Prediction ERL: 1590
GT TRL: 1742560, Prediction TRL: 1522548
Omitted RL: 46502, Merged RL: 72981, Split RL: 100528
Starting voxel evaluation methods for waterz0.50-48nm-crop2gt-0014 against gt-48nm-60slices-0014
Original VI split, VI merge: 0.071742, 0.174443
After fixing 44 merges in GT flagged by skeleton analysis:


In [5]:
# run error analysis of unlinked target block
tb_vox_eval = VoxEval(gt_tb_name, pred_tb_name, stage="filtered")
tb_vox_eval.run_fullsuite(iou_max=0.6, hist_segs=10, 
                          thresh_miss=t_om, overwrite_prev=False)
tb_skel_eval = SkelEval(gt_tb_name, pred_tb_name, dsmpl_res=(80,80,80), 
	                 t_om=t_om, t_m=t_m, t_s=t_s, 
	                 include_zero_split=False, include_zero_merge=True,
	                 stage="filtered", overwrite_prev=True)
tb_skel_eval.merge_oracle()
tb_skel_eval.split_oracle()

Starting voxel evaluation methods for waterz0.50-48nm-crop2gt-0074 against gt-48nm-60slices-0074
Missing object results loaded
VI loaded
IoU results loaded
delta_VI results loaded
Delta VI histogram plots generated and saved
Starting error analysis of waterz0.50-48nm-crop2gt-0074 against skeletons of gt-48nm-60slices-0074
Warning! Labels in predicted segmentation are not consecutive. Proceeding anyway
Starting evaluation of 5576 labels in 60x540x489 predicted segmentation against 1001 GT skeletons
Using error thresholds: t_om=0.90, t_m=0.50, t_s=0.80
Skeleton evaluation time: 3.76012396812
Results:
54 omissions, 8 merges, 175 splits, 11 hybrid, 753 correct
GT ERL: 2299, Prediction ERL: 1478
GT TRL: 1808707, Prediction TRL: 1467614
Omitted RL: 128787, Merged RL: 67471, Split RL: 144833
Starting voxel evaluation methods for waterz0.50-48nm-crop2gt-0074 against gt-48nm-60slices-0074
Original VI split, VI merge: 0.096858, 0.118013
After fixing 65 merges in GT flagged by skeleton analysis:


## Superblock error analysis

In [8]:
# generate GT superblock for error analysis
gt_superblock_type = 'gt48nm-linear-superblock'
n_blocks = 2
gt_superblock_name = gt_superblock_type + '-%dblocks'%(n_blocks)

gt8nm_file = "/home/srujanm/cerebellum/data/vol1/gt8nm_pf-align1_1k_crop.h5" # location of GT on disk
gt_superblock_seg = SegPrep(gt_superblock_name, resolution)
gt_superblock_seg.read(gt8nm_file, 'main', dsmpl=(1,6,6),
                       block_lims=((0,n_blocks*block_size),(None,None),(None,None)))
# gt_superblock_seg.gen_bboxes()
gt_superblock_seg.read_bboxes() # read bboxes generated above
gt_superblock_seg.relabel(use_bboxes=True)
gt_superblock_seg.write()

# assuming relabeling has already been done proceed to skeletonize
from cerebellum.skeletonize import gen_skeletons
gen_skeletons(gt_superblock_name, resolution, overwrite_prev=True)

Starting relabeling of 1003 objects
Relabeling time: 0.427247
Starting skeletonization of gt48nm-linear-superblock-2blocks
Downsampling to resolution (80, 80, 80) in 0.6146068573 seconds
Topological thinning time for (80, 80, 80): 9.49117493629
Endpoint vector time for (80, 80, 80): 2.15331697464
Edge finding time for (80, 80, 80): 0.4236369133


In [9]:
# prepare predicted segmentation superblock for error analysis
iou_thresh = 0.4
pred_superblock_type = 'waterz%.2f-48nm-crop2gt-linear-superblock-iou-%.2f'%(wz_thresh, iou_thresh)
pred_superblock_name = pred_superblock_type + '-%dblocks'%(n_blocks)
pred_superblock_seg = SegPrep(pred_superblock_name, resolution)
pred_superblock_seg.read('./superblocks/'+pred_superblock_name+'.h5', 'main')
pred_superblock_seg.write()

In [10]:
# run error analysis
superblock_vox_eval = VoxEval(gt_superblock_name, pred_superblock_name)
superblock_vox_eval.find_misses(thresh_miss=t_om)
superblock_vox_eval.find_vi()
superblock_vox_eval.get_vi()
superblock_skel_eval = SkelEval(gt_superblock_name, pred_superblock_name, dsmpl_res=(80,80,80), 
                                 t_om=t_om, t_m=t_m, t_s=t_s, 
                                 include_zero_split=False, include_zero_merge=True,
                                 overwrite_prev=True)
superblock_skel_eval.merge_oracle()
superblock_skel_eval.split_oracle()

Starting voxel evaluation methods for waterz0.50-48nm-crop2gt-linear-superblock-iou-0.40-2blocks against gt48nm-linear-superblock-2blocks
Found 3 GT objects completely missing
VI split, VI merge: 0.257888, 0.935681
Error retrieving VI because VI calculation has not been run yet
Starting error analysis of waterz0.50-48nm-crop2gt-linear-superblock-iou-0.40-2blocks against skeletons of gt48nm-linear-superblock-2blocks
Warning! Labels in predicted segmentation are not consecutive. Proceeding anyway
Starting evaluation of 7733 labels in 120x540x489 predicted segmentation against 1003 GT skeletons
Using error thresholds: t_om=0.90, t_m=0.50, t_s=0.80
Skeleton evaluation time: 5.40400791168
Results:
4 omissions, 35 merges, 298 splits, 31 hybrid, 635 correct
GT ERL: 4506, Prediction ERL: 2958
GT TRL: 4258598, Prediction TRL: 3283719
Omitted RL: 13420, Merged RL: 361729, Split RL: 599728
Starting voxel evaluation methods for waterz0.50-48nm-crop2gt-linear-superblock-iou-0.40-2blocks against gt4